<a href="https://colab.research.google.com/github/santiagorey16/PROGCOM-A/blob/main/juego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import messagebox
import random
import os

# ===================== RUTAS DE RECURSOS =====================
IMG_FOLDER = "img"
PLAYER_IMG_FILE = os.path.join(IMG_FOLDER, "nino.png")          # imagen del niño
BG_IMG_FILE = os.path.join(IMG_FOLDER, "barrio_noche.png")      # imagen del barrio

# ===================== SONIDO DE FONDO =======================
try:
    import winsound   # solo existe en Windows
except ImportError:
    winsound = None


def start_music():
    """
    Música / sonido de fondo en loop.
    Uso un sonido del sistema de Windows para no depender de archivos externos.
    """
    if winsound is None:
        return
    winsound.PlaySound("SystemExit",
                       winsound.SND_ALIAS | winsound.SND_ASYNC | winsound.SND_LOOP)


def stop_music():
    if winsound is None:
        return
    winsound.PlaySound(None, 0)


# ===================== ENTIDADES DEL JUEGO ===================
class Player:
    def __init__(self, x, y, sprite=None):
        self.x = x
        self.y = y
        self.r = 20
        self.speed = 6
        self.sprite = sprite
        if self.sprite is not None:
            self.w = self.sprite.width()
            self.h = self.sprite.height()
        else:
            self.w = self.h = self.r * 2

    def update(self, keys, width, height):
        if keys.get("Left"):
            self.x -= self.speed
        if keys.get("Right"):
            self.x += self.speed
        if keys.get("Up"):
            self.y -= self.speed
        if keys.get("Down"):
            self.y += self.speed

        # limitar a la pantalla
        self.x = max(self.r, min(width - self.r, self.x))
        self.y = max(self.r, min(height - self.r, self.y))

    def draw(self, canvas):
        if self.sprite is not None:
            canvas.create_image(self.x, self.y, image=self.sprite)
        else:
            canvas.create_oval(
                self.x - self.r, self.y - self.r,
                self.x + self.r, self.y + self.r,
                fill="#ffdd55", outline=""
            )


class Candy:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.r = 8

    def draw(self, canvas):
        canvas.create_oval(
            self.x - self.r, self.y - self.r,
            self.x + self.r, self.y + self.r,
            fill="#ff00ff", outline=""
        )
        canvas.create_line(self.x - self.r - 4, self.y,
                           self.x - self.r + 1, self.y,
                           fill="white", width=2)
        canvas.create_line(self.x + self.r - 1, self.y,
                           self.x + self.r + 4, self.y,
                           fill="white", width=2)


class Bullet:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.r = 4
        self.speed = 9

    def update(self):
        self.y -= self.speed

    def draw(self, canvas):
        canvas.create_oval(
            self.x - self.r, self.y - self.r,
            self.x + self.r, self.y + self.r,
            fill="#00ffff", outline=""
        )


class Enemy:
    def __init__(self, enemy_type, width, height, speed_factor=1.0):
        self.type = enemy_type  # "ghost" o "zombie"
        self.r = 18 if enemy_type == "ghost" else 22
        base_speed = 2.2 if enemy_type == "ghost" else 1.4
        self.speed = base_speed * speed_factor
        self.hp = 1 if enemy_type == "ghost" else 2

        side = random.randint(0, 2)
        if side == 0:        # arriba
            self.x = random.randint(0, width)
            self.y = -30
        elif side == 1:      # izquierda
            self.x = -30
            self.y = random.randint(0, int(height * 0.8))
        else:                # derecha
            self.x = width + 30
            self.y = random.randint(0, int(height * 0.8))

    def update(self, player):
        dx = player.x - self.x
        dy = player.y - self.y
        dist = (dx ** 2 + dy ** 2) ** 0.5 or 1
        self.x += self.speed * dx / dist
        self.y += self.speed * dy / dist

    def draw(self, canvas):
        if self.type == "ghost":
            canvas.create_arc(
                self.x - self.r, self.y - self.r,
                self.x + self.r, self.y + self.r,
                start=0, extent=180, fill="#ccccff", outline=""
            )
            canvas.create_rectangle(
                self.x - self.r, self.y,
                self.x + self.r, self.y + self.r,
                fill="#ccccff", outline=""
            )
        else:
            canvas.create_oval(
                self.x - self.r, self.y - self.r,
                self.x + self.r, self.y + self.r,
                fill="#55aa55", outline=""
            )
        # ojos
        canvas.create_oval(self.x - 7, self.y - 5, self.x - 3, self.y - 1, fill="black")
        canvas.create_oval(self.x + 3, self.y - 5, self.x + 7, self.y - 1, fill="black")


# ===================== JUEGO PRINCIPAL ======================
class HalloweenGame:
    def __init__(self, root):
        self.root = root
        self.width = 800
        self.height = 500

        root.title("Noche de Dulces – Fantasmas y Zombies")
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg="black")
        self.canvas.pack()

        # HUD
        self.label_info = tk.Label(
            root,
            text="",
            fg="white",
            bg="black",
            font=("Segoe UI", 10)
        )
        self.label_info.pack(fill="x")

        # imágenes
        self.bg_image = None
        self.player_image = None
        self.load_images()

        # estado
        self.state = "menu"     # "menu", "start", "playing", "game_over"
        self.difficulty = None  # "easy", "medium", "hard"

        self.keys = {}
        self.player = Player(self.width // 2, self.height - 70, self.player_image)
        self.candies = []
        self.enemies = []
        self.bullets = []
        self.score = 0
        self.wave = 1
        self.shoot_cooldown = 0

        # parámetros según dificultad
        self.enemy_speed_factor = 1.0
        self.enemy_count_factor = 1.0

        # eventos
        self.root.bind("<KeyPress>", self.on_key_press)
        self.root.bind("<KeyRelease>", self.on_key_release)
        self.canvas.bind("<Button-1>", self.on_click)

        self.create_difficulty_menu()
        self.update_hud()
        start_music()
        self.game_loop()

    # ---------- carga de imágenes ----------
    def load_images(self):
        if os.path.exists(BG_IMG_FILE):
            self.bg_image = tk.PhotoImage(file=BG_IMG_FILE)
        if os.path.exists(PLAYER_IMG_FILE):
            self.player_image = tk.PhotoImage(file=PLAYER_IMG_FILE)

    # ---------- menú de dificultad ----------
    def create_difficulty_menu(self):
        self.menu_frame = tk.Frame(self.canvas, bg="black")
        self.menu_frame.place(relx=0.5, rely=0.5, anchor="center")

        lbl = tk.Label(
            self.menu_frame,
            text="Elige dificultad",
            bg="black",
            fg="white",
            font=("Segoe UI", 16, "bold")
        )
        lbl.pack(pady=10)

        btn_easy = tk.Button(self.menu_frame, text="Fácil", width=12,
                             command=lambda: self.set_difficulty("easy"))
        btn_medium = tk.Button(self.menu_frame, text="Medio", width=12,
                               command=lambda: self.set_difficulty("medium"))
        btn_hard = tk.Button(self.menu_frame, text="Difícil", width=12,
                             command=lambda: self.set_difficulty("hard"))

        btn_easy.pack(pady=5)
        btn_medium.pack(pady=5)
        btn_hard.pack(pady=5)

    def set_difficulty(self, level):
        self.difficulty = level
        if level == "easy":
            self.enemy_speed_factor = 0.8
            self.enemy_count_factor = 0.7
        elif level == "medium":
            self.enemy_speed_factor = 1.0
            self.enemy_count_factor = 1.0
        else:  # hard
            self.enemy_speed_factor = 1.4
            self.enemy_count_factor = 1.3

        self.menu_frame.destroy()
        self.reset_game()

    # ---------- entrada ----------
    def on_key_press(self, event):
        self.keys[event.keysym] = True
        if self.state == "start" and event.keysym == "space":
            self.start_game()
        if self.state == "game_over" and event.keysym == "space":
            self.reset_game()

    def on_key_release(self, event):
        self.keys[event.keysym] = False

    def on_click(self, event):
        if self.state == "start":
            self.start_game()
        elif self.state == "game_over":
            self.reset_game()

    # ---------- lógica ----------
    def reset_game(self):
        self.state = "start"
        self.player = Player(self.width // 2, self.height - 70, self.player_image)
        self.candies = []
        self.enemies = []
        self.bullets = []
        self.score = 0
        self.wave = 1

        base_candies = 5
        base_enemies = 3
        self.spawn_candies(int(base_candies * self.enemy_count_factor))
        self.spawn_enemies(int(base_enemies * self.enemy_count_factor))
        self.update_hud()

    def start_game(self):
        self.state = "playing"
        self.update_hud()

    def spawn_candies(self, n):
        for _ in range(n):
            x = random.randint(40, self.width - 40)
            y = random.randint(60, self.height - 120)
            self.candies.append(Candy(x, y))

    def spawn_enemies(self, n):
        for _ in range(n):
            t = "ghost" if random.random() < 0.5 else "zombie"
            self.enemies.append(Enemy(t, self.width, self.height,
                                      speed_factor=self.enemy_speed_factor))

    def update_hud(self):
        if self.state == "menu":
            txt = "Elige dificultad para comenzar"
        elif self.state == "start":
            txt = "Pulsa ESPACIO o clic para iniciar | Flechas: moverse, Barra: disparar"
        elif self.state == "playing":
            txt = (f"Dulces: {self.score}   Oleada: {self.wave}   "
                   f"Dificultad: {self.difficulty}")
        else:
            txt = "GAME OVER – ESPACIO o clic para reiniciar"
        self.label_info.config(text=txt)

    def distance(self, x1, y1, x2, y2):
        return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

    def game_loop(self):
        self.update()
        self.draw()
        self.root.after(30, self.game_loop)  # ~33 FPS

    def update(self):
        if self.state != "playing":
            return

        self.player.update(self.keys, self.width, self.height)

        # disparo
        if self.shoot_cooldown > 0:
            self.shoot_cooldown -= 1
        if self.keys.get("space") or self.keys.get("Space"):
            if self.shoot_cooldown == 0:
                self.bullets.append(Bullet(self.player.x, self.player.y - self.player.r))
                self.shoot_cooldown = 10

        # balas
        for b in self.bullets:
            b.update()
        self.bullets = [b for b in self.bullets if b.y + b.r > 0]

        # enemigos
        for e in self.enemies:
            e.update(self.player)

        # colisión balas-enemigos
        new_enemies = []
        for e in self.enemies:
            muerto = False
            for b in list(self.bullets):
                if self.distance(b.x, b.y, e.x, e.y) < e.r + b.r:
                    self.bullets.remove(b)
                    e.hp -= 1
                    if e.hp <= 0:
                        muerto = True
                        self.score += 5
                    break
            if not muerto:
                new_enemies.append(e)
        self.enemies = new_enemies

        # 💀 colisión jugador-enemigo → muerte instantánea
        for e in self.enemies:
            if self.distance(self.player.x, self.player.y, e.x, e.y) < self.player.r + e.r:
                self.game_over()
                return

        # jugador-dulces
        new_candies = []
        for c in self.candies:
            if self.distance(self.player.x, self.player.y, c.x, c.y) < self.player.r + c.r:
                self.score += 1
            else:
                new_candies.append(c)
        self.candies = new_candies

        # nueva oleada
        if not self.candies:
            self.wave += 1
            extra = self.wave
            self.spawn_candies(int((3 + extra) * self.enemy_count_factor))
            self.spawn_enemies(int((2 + extra // 2) * self.enemy_count_factor))

        self.update_hud()

    def game_over(self):
        self.state = "game_over"
        self.update_hud()
        messagebox.showinfo(
            "Fin del juego",
            f"Game Over\nDulces recogidos: {self.score}\nOleada alcanzada: {self.wave}"
        )

    def draw(self):
        self.canvas.delete("all")

        # fondo del barrio de noche
        if self.bg_image is not None:
            self.canvas.create_image(self.width // 2, self.height // 2, image=self.bg_image)
        else:
            self.canvas.create_rectangle(0, 0, self.width, self.height,
                                         fill="#000022", outline="")

        # suelo
        self.canvas.create_rectangle(
            0, self.height - 40, self.width, self.height,
            fill="#111111", outline=""
        )

        for c in self.candies:
            c.draw(self.canvas)
        for b in self.bullets:
            b.draw(self.canvas)
        for e in self.enemies:
            e.draw(self.canvas)
        self.player.draw(self.canvas)

        if self.state == "start":
            self.canvas.create_text(
                self.width // 2, self.height // 2,
                text="NOCHE DE DULCES\n\nRecoge caramelos y dispara\n"
                     "a fantasmas y zombies",
                fill="orange", font=("Segoe UI", 16, "bold"), justify="center"
            )
        elif self.state == "game_over":
            self.canvas.create_text(
                self.width // 2, self.height // 2 + 60,
                text="Pulsa ESPACIO o haz clic para reiniciar",
                fill="red", font=("Segoe UI", 12, "bold")
            )

    def on_close(self):
        stop_music()
        self.root.destroy()


# ===================== ARRANQUE ===========================
if __name__ == "__main__":
    root = tk.Tk()
    game = HalloweenGame(root)
    root.protocol("WM_DELETE_WINDOW", game.on_close)
    root.mainloop()